In [ ]:
# Importamos las librerías, si no están instaladas, realizar instalación con pip install en terminal
import cv2 # CV, libreria que sirve para procesar imágenes
import pytesseract # OCR, sirve para extraer texto de imágenes
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer # convierte el texto en una matriz
from sklearn.linear_model import LogisticRegression # ML, para predicción
import numpy as np

# Configuramos Tesseract (aquí colocar la ruta donde se instaló)
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Ingresamos una función para extraer texto usando visión computacional
def extraer_texto(imagen_path):
    imagen = cv2.imread("C:\Ciencia de Datos\Python\Imagenes\etiqueta_danada.jpg") # especificar la ruta
    gris = cv2.cvtColor(imagen, cv2.COLOR_BGR2GRAY)
    texto = pytesseract.image_to_string(gris, config='--psm 6') # para extaer el texto
    return texto.strip()

# Preparamos el modelo de Machine Learning para predecir caraceters faltantes
def entrenar_modelo():
    datos = {
        "Etiqueta": ["GBR-120", "GBR-121", "GBR-122", "GBR-128", "GBR-128", "GBR-128", "GBR-123", "GBR-129", "GBR-124"],
        "Ultimo_Caracter": ["0", "1", "2", "8", "8", "8", "3", "9", "4"] # datos de entrenamiento correctos
    }
    df = pd.DataFrame(datos)
    # Convertimos las etiquetas en una representación numérica basada en caracteres
    vectorizador = CountVectorizer(analyzer='char', ngram_range=(5, 5))
    X = vectorizador.fit_transform(df["Etiqueta"])
    y = df["Ultimo_Caracter"]
    # Utilizamos el modelo ML (regresión logística) para predecir los valores faltantes
    modelo = LogisticRegression(max_iter=1000)
    modelo.fit(X, y) # entrenamos el modelo
    return modelo, vectorizador

# Utilizamos una función para completar el texto si falta un carácter usando ML
def completar_texto(texto, modelo, vectorizador):
    print(f"Texto extraído: {texto}") # mostramos el texto detectado
    if "_" in texto:  # verificamos si hay un carácter faltante, se representa como _
        X_prueba = vectorizador.transform([texto]) 
        prediccion = modelo.predict(X_prueba) # el modelo predice el caracter faltante
        texto = texto.replace("_", prediccion[0])  # completamos el carácter faltante
        print(f"Carácter faltante completado automáticamente: {prediccion[0]}")
    return texto

# Cargamos y procesamos la imagen
imagen_path = "etiqueta_danada.jpg"
texto_extraido = extraer_texto(imagen_path)

# Entrenamos el modelo
modelo, vectorizador = entrenar_modelo()

# Completamos el texto si falta un carácter
texto_final = completar_texto(texto_extraido, modelo, vectorizador)

# Mostramos el resultado final
print(f"Texto final corregido: {texto_final}")

Texto extraído: GBR-12_
Carácter faltante completado automáticamente: 8
Texto final corregido: GBR-128
